<span style="color: black; font-weight: bold; font-size: 50px;">Récuperation de la liste des entreprises - Bitrix 24(YAV)</span>

In [1]:
import time
import configparser
import requests
import pandas as pd

def CrmBitrix(content='', params=[], result='', pause_duration=60, max_retries=3, batch_limit=30000):
    config = configparser.ConfigParser()
    config.read('Config.ini')

    if 'BITRIX' not in config:
        raise ValueError('Section BITRIX manquante dans le fichier \'config.ini\'')

    BITRIX_DOMAIN = config.get('BITRIX', 'BITRIX_DOMAIN')
    BITRIX_TOKEN = config.get('BITRIX', 'BITRIX_TOKEN')

    if not BITRIX_DOMAIN or not BITRIX_TOKEN:
        raise ValueError('Domaine et/ou token Bitrix manquants dans le fichier \'config.ini\'')

    params_str = '&'.join(params)
    BitrixUrl = f'https://{BITRIX_DOMAIN}/rest/1/{BITRIX_TOKEN}/{content}?{params_str}&start='

    listId, listTITLE, listEmails, listNumeros, listDATE_CREATE = [], [], [], [], []
    listActivite1, listActivite2, listActivite3 = [], [], []
    listAdresse, listCodePostale, listVille, listSiret, listidd, listphones, listProvenance = [], [], [], [], [], [], []
    listCOMMENTS, listWeb, listNombreapprenant, listNombreformateur, listSoustraitance=[], [], [], [], []

    start = 0
    total_records = 0

    while True:
        url = BitrixUrl + str(start)
        
        for attempt in range(max_retries):
            try:
                response = requests.get(url)
                response.raise_for_status()
                break  
            except requests.exceptions.HTTPError as e:
                if response.status_code == 429:
                    wait_time = pause_duration * (2 ** attempt)
                    print(f'Rate limit exceeded. Retrying in {wait_time} seconds...')
                    time.sleep(wait_time)
                else:
                    raise ValueError(f'Erreur lors de la récupération depuis Bitrix : {e}')
            except requests.exceptions.RequestException as e:
                raise ValueError(f'Erreur lors de la récupération depuis Bitrix : {e}')

        data = response.json()

        if 'result' in data:
            contacts = data['result']
            for contact in contacts:
                listId.append(contact.get('ID', ''))
                listTITLE.append(contact.get('TITLE', ''))
                listEmails.append(contact.get('EMAIL', [{}])[0].get('VALUE', ''))
                listNumeros.append(contact.get('PHONE', [{}])[0].get('VALUE', ''))
                listWeb.append(contact.get('WEB', [{}])[0].get('VALUE', ''))
                listphones.append(contact.get('UF_CRM_1741080141', ''))
                listDATE_CREATE.append(contact.get('DATE_CREATE', ''))
                listActivite1.append(contact.get('UF_CRM_1686295056', ''))
                listActivite2.append(contact.get('UF_CRM_1686295079', ''))
                listActivite3.append(contact.get('UF_CRM_1686295100', ''))
                listAdresse.append(contact.get('UF_CRM_1686295131', ''))
                listCodePostale.append(contact.get('UF_CRM_1686295169', ''))
                listVille.append(contact.get('UF_CRM_1686295194', ''))
                listSiret.append(contact.get('UF_CRM_1686295022', ''))
                listNombreapprenant.append(contact.get('UF_CRM_1686295269', ''))
                listNombreformateur.append(contact.get('UF_CRM_1686295348', ''))
                listSoustraitance.append(contact.get('UF_CRM_1686295381', ''))
                listProvenance.append(contact.get('UF_CRM_1723467653', ''))
                listCOMMENTS.append(contact.get('COMMENTS', ''))
                listidd.append(contact.get('UF_CRM_1722346888', ''))
                

            total_records += len(contacts) 

            if total_records >= batch_limit:
                print(f"Atteint {batch_limit} enregistrements, pause de 5 minutes...")
                time.sleep(300) 
                total_records = 0 

        if 'next' in data:
            start = data['next']
        else:
            break

    df_btx = pd.DataFrame({
        'ID': listId,
        'CompanyId': listId,
        'nom': listTITLE,
        "Email": listEmails,
        "Crée": listDATE_CREATE,
        "Mobile": listNumeros,
        "Phones": listphones,
        "Activite1": listActivite1,
        "Activite2": listActivite2,
        "Activite3": listActivite3,
        "Adresse": listAdresse,
        "CodePostale": listCodePostale,
        "Ville": listVille,
        "Siret": listSiret,
        "Nombre apprenants": listNombreapprenant,
        "Nombre de Formateurs": listNombreformateur,
        "Nombre de formateur sous traitance": listSoustraitance,
        "Provenance": listProvenance,
        "Commentaires": listCOMMENTS,
        "Site web": listWeb,
        "IDS": listidd
    })

    return df_btx

dfb = CrmBitrix(
    content='crm.company.list.json',
    params=[
        'select[]=ID',
        'select[]=TITLE',
        'select[]=EMAIL',
        'select[]=PHONE',
        'select[]=WEB',
        'select[]=UF_CRM_1741080141',
        'select[]=DATE_CREATE',
        'select[]=UF_CRM_1686295056',
        'select[]=UF_CRM_1686295079',
        'select[]=UF_CRM_1686295100',
        'select[]=UF_CRM_1686295131',
        'select[]=UF_CRM_1686295169',
        'select[]=UF_CRM_1686295194', 
        'select[]=UF_CRM_1686295022',
        'select[]=UF_CRM_1686295269',
        'select[]=UF_CRM_1686295348',
        'select[]=UF_CRM_1686295381',
        'select[]=UF_CRM_1723467653',
        'select[]=COMMENTS',
        'select[]=UF_CRM_1722346888',
    ],
    result="title"
)
dfb


Atteint 30000 enregistrements, pause de 5 minutes...
Atteint 30000 enregistrements, pause de 5 minutes...
Atteint 30000 enregistrements, pause de 5 minutes...


,ID,CompanyId,nom,Email,Crée,Mobile,Phones,Activite1,Activite2,Activite3,...,CodePostale,Ville,Siret,Nombre apprenants,Nombre de Formateurs,Nombre de formateur sous traitance,Provenance,Commentaires,Site web,IDS
0,4,4,0CEAN INDIEN INFORMATIQUE SARL,contact@0ceanindienifnormatioque.cfr,2023-06-09T10:32:19+03:00,,None,Spécialités plurivalentes des échanges et de l...,"Informatique, traitement de l'information, rés...","Enseignement, formation",...,None,None,38452004500046,225,2,0,None,None,,140
1,8,8,0'LISTY,,2023-06-09T10:32:19+03:00,,None,Autres...,None,None,...,97227,STE ANNE,88943425400013,None,1,None,None,None,,140
2,10,10,1000 ET DEUX IDEES,,2023-06-09T10:32:20+03:00,,None,Journalisme et communication,None,None,...,35360,BOISGERVILLY,51202144500014,0,1,103,None,None,,140
3,16,16,1001 TAILLES,,2023-06-09T10:32:20+03:00,,None,"Habillement (y compris mode, couture)",None,None,...,None,None,52815239000019,1,1,0,None,None,,140
4,18,18,100POUR100COM,,2023-06-09T10:32:20+03:00,,None,None,None,None,...,None,None,84444399400018,0,1,103,None,None,,140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102287,520220,520220,WEBER FRANCE,,2025-05-08T10:14:43+03:00,,None,Formations générales,None,None,...,None,None,48808246200034,None,None,None,None,None,,140
102288,520222,520222,WIKLOG,,2025-05-08T10:14:43+03:00,,None,"Informatique, traitement de l'information, rés...",None,None,...,None,None,83932390400038,None,None,None,None,None,,140
102289,520224,520224,WOUNDI,,2025-05-08T10:14:43+03:00,,None,Sciences naturelles (Biologie-Géologie),None,None,...,97410,ST PIERRE,97894667100010,None,None,None,None,None,,140
102290,520226,520226,YELLOW MONKEYS,,2025-05-08T10:14:44+03:00,,None,Journalisme et communication,None,None,...,None,None,84812905200025,None,None,None,None,None,,140


In [30]:
dfb.to_csv("company.csv", index=False)
dfb.drop_duplicates()
dfb

,ID,CompanyId,nom,Email,Crée,Mobile,Phones,Activite1,Activite2,Activite3,...,CodePostale,Ville,Siret,Nombre apprenants,Nombre de Formateurs,Nombre de formateur sous traitance,Provenance,Commentaires,Site web,IDS
0,4,4,0CEAN INDIEN INFORMATIQUE SARL,contact@0ceanindienifnormatioque.cfr,2023-06-09T10:32:19+03:00,,None,Spécialités plurivalentes des échanges et de l...,"Informatique, traitement de l'information, rés...","Enseignement, formation",...,None,None,38452004500046,225,2,0,None,None,,140
1,8,8,0'LISTY,,2023-06-09T10:32:19+03:00,,None,Autres...,None,None,...,97227,STE ANNE,88943425400013,None,1,None,None,None,,140
2,10,10,1000 ET DEUX IDEES,,2023-06-09T10:32:20+03:00,,None,Journalisme et communication,None,None,...,35360,BOISGERVILLY,51202144500014,0,1,103,None,None,,140
3,16,16,1001 TAILLES,,2023-06-09T10:32:20+03:00,,None,"Habillement (y compris mode, couture)",None,None,...,None,None,52815239000019,1,1,0,None,None,,140
4,18,18,100POUR100COM,,2023-06-09T10:32:20+03:00,,None,None,None,None,...,None,None,84444399400018,0,1,103,None,None,,140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99557,510850,510850,Madame MELISSA SAMUT,,2025-04-08T11:25:26+03:00,,None,"Coiffure, esthétique et autres spécialités des...",None,None,...,57950,MONTIGNY LES METZ,88821318800038,None,None,None,None,None,,140
99558,510852,510852,Madame MORGANE LAMBOUR,,2025-04-08T11:25:26+03:00,,None,Développement des capacités comportementales e...,None,None,...,80880,ST QUENTIN LA MOTTE CROIX AU BAILLY,92969041000022,None,None,None,None,None,,140
99559,510854,510854,Madame MORGAN MASIP,,2025-04-08T11:25:26+03:00,,None,"Droit, sciences politiques",None,None,...,None,None,98412835500018,None,None,None,None,None,,140
99560,510856,510856,Madame MURIEL RUGGIERI,,2025-04-08T11:25:26+03:00,,None,"Comptabilité, gestion",None,None,...,33240,CUBZAC LES PONTS,41037999400024,None,None,None,None,None,,140


In [31]:
import pandas as pd
import requests
defaultCrmProspectListUrl = ""

listUrls = []
listeId = []
listID = []
listValuesource = []
listIDd = []

datas = {}

responses = requests.get(defaultCrmProspectListUrl)
        
def testedv():
    if responses.status_code == 200:
        datas = responses.json()
        companyfield = datas['result'][10]['LIST']

        for source in companyfield:
            listID.append(source['ID'])
            listValuesource.append(source['VALUE'])
           
            

            df_btx = pd.DataFrame(list(zip(listID, listValuesource)),
                              columns = ["IDS",'Statut'])

        return df_btx

In [32]:
dfdd = testedv()
dfdd

,IDS,Statut
0,140,Pas Qualiopi
1,142,Qualiopi
2,146,Qualiopi perdu
3,158,NRP + SMS
4,160,NRP + Email Personnalisé
5,162,NRP + Message Vocal
6,164,Contact Obtenu
7,144,Portage via un concurrent
8,166,Pas intéressé
9,168,Fermé


In [33]:
df_Bitrixyav = dfb.merge(dfdd, how='left')
df_Bitrixyav.drop_duplicates()
df_Bitrixyav.to_csv('companyyavlist.csv', index=False, encoding='utf-8')
df_Bitrixyav

,ID,CompanyId,nom,Email,Crée,Mobile,Phones,Activite1,Activite2,Activite3,...,Ville,Siret,Nombre apprenants,Nombre de Formateurs,Nombre de formateur sous traitance,Provenance,Commentaires,Site web,IDS,Statut
0,4,4,0CEAN INDIEN INFORMATIQUE SARL,contact@0ceanindienifnormatioque.cfr,2023-06-09T10:32:19+03:00,,None,Spécialités plurivalentes des échanges et de l...,"Informatique, traitement de l'information, rés...","Enseignement, formation",...,None,38452004500046,225,2,0,None,None,,140,Pas Qualiopi
1,8,8,0'LISTY,,2023-06-09T10:32:19+03:00,,None,Autres...,None,None,...,STE ANNE,88943425400013,None,1,None,None,None,,140,Pas Qualiopi
2,10,10,1000 ET DEUX IDEES,,2023-06-09T10:32:20+03:00,,None,Journalisme et communication,None,None,...,BOISGERVILLY,51202144500014,0,1,103,None,None,,140,Pas Qualiopi
3,16,16,1001 TAILLES,,2023-06-09T10:32:20+03:00,,None,"Habillement (y compris mode, couture)",None,None,...,None,52815239000019,1,1,0,None,None,,140,Pas Qualiopi
4,18,18,100POUR100COM,,2023-06-09T10:32:20+03:00,,None,None,None,None,...,None,84444399400018,0,1,103,None,None,,140,Pas Qualiopi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99557,510850,510850,Madame MELISSA SAMUT,,2025-04-08T11:25:26+03:00,,None,"Coiffure, esthétique et autres spécialités des...",None,None,...,MONTIGNY LES METZ,88821318800038,None,None,None,None,None,,140,Pas Qualiopi
99558,510852,510852,Madame MORGANE LAMBOUR,,2025-04-08T11:25:26+03:00,,None,Développement des capacités comportementales e...,None,None,...,ST QUENTIN LA MOTTE CROIX AU BAILLY,92969041000022,None,None,None,None,None,,140,Pas Qualiopi
99559,510854,510854,Madame MORGAN MASIP,,2025-04-08T11:25:26+03:00,,None,"Droit, sciences politiques",None,None,...,None,98412835500018,None,None,None,None,None,,140,Pas Qualiopi
99560,510856,510856,Madame MURIEL RUGGIERI,,2025-04-08T11:25:26+03:00,,None,"Comptabilité, gestion",None,None,...,CUBZAC LES PONTS,41037999400024,None,None,None,None,None,,140,Pas Qualiopi


<span style="color: black; font-weight: bold; font-size: 50px;">La liste des entreprises(YAV) -> Comptes (Freshworks)</span>

In [ ]:
import csv
import requests
import json
import time

API_KEY_FRESHWORK = 'Z2DYl5CbmvdNqhLBLm_-Ww'
DOMAIN_FRESHWORK = 'yavfr'
BASE_URL = f'https://{DOMAIN_FRESHWORK}.myfreshworks.com/crm/sales/'

HEADERS = {
    'Content-Type': 'application/json',
    'Authorization': f'Token token={API_KEY_FRESHWORK}'
}

BATCH_SIZE = 900
SLEEP_DURATION = 3600

def create_account(account_data):
    """Créer un compte (account) dans Freshworks."""
    print(f"\nCréation du compte : {account_data['name']} (cf_id_bitrix={account_data['custom_field']['cf_id_bitrix']})")
    
    url = f"{BASE_URL}api/sales_accounts"
    response = requests.post(url, headers=HEADERS, json={'sales_account': account_data})

    if response.status_code in [200, 201]:
        account_id = response.json()['sales_account']['id']
        print(f"Compte créé : ID = {account_id}")
        return account_id
    else:
        print(f"Erreur pour {account_data['custom_field']['cf_id_bitrix']} : {response.status_code} - {response.text}")
        return None

def extract_account_info(row):
    """Mapper les colonnes du CSV vers les champs Freshworks (standard + custom_field)."""
    return {
        "name": row.get("nom", "Nom inconnu").strip(),
        "phone": row.get("Phones", "").strip(),
        "website": row.get("Site web", "").strip(),
        "address": row.get("Adresse", "").strip(),
        "city": row.get("Ville", "").strip(),
        "zipcode": row.get("CodePostale", "").strip(),
        "custom_field": {
            "cf_id_bitrix": row.get("ID", "").strip(),
            "cf_date_de_creation": row.get("Crée", "").strip(),
            "cf_siret": row.get("Siret", "").strip(),
            "cf_actiit_1": row.get("Activite1", "").strip(),
            "cf_activit_2": row.get("Activite2", "").strip(),
            "cf_activit_3": row.get("Activite3", "").strip(),
            "cf_nombre_apprenants": row.get("Nombre apprenants", "").strip(),
            "cf_nombre_formateurs": row.get("Nombre de Formateurs", "").strip(),
            "cf_nombre_apprenant_sous_traitance": row.get("Nombre de formateur sous traitance", "").strip(),
            "cf_provenance": row.get("Provenance", "").strip(),
            "cf_commentaire": row.get("Commentaires", "").strip(),
            "cf_email": row.get("Email", "").strip().lower(),
            "cf_statut_entreprise": row.get("Statut", "").strip()
        }
    }

def main():
    """Lecture CSV, mapping, envoi vers Freshworks en batchs."""
    csv_file = 'companyyavlist.csv'

    with open(csv_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        accounts_data = list(reader)

    total_accounts = len(accounts_data)
    print(f"\nTotal de comptes à traiter : {total_accounts}")

    processed = 0
    batch = 1

    for i, row in enumerate(accounts_data):
        if processed != 0 and processed % BATCH_SIZE == 0:
            print(f"\nBatch {batch} terminé ({processed} comptes). Pause de {SLEEP_DURATION // 60} minutes...")
            batch += 1
            time.sleep(SLEEP_DURATION)

        account_info = extract_account_info(row)
        create_account(account_info)

        processed += 1
        time.sleep(1)

    print(f"\nTous les comptes ont été traités ({processed} au total).")


if __name__ == "__main__":
    main()


<span style="color: black; font-weight: bold; font-size: 50px;">Récuperation de la liste des contacts (Yav) </span>

In [ ]:
import time
import configparser
import requests
import pandas as pd

def CrmBitrix(content='', params=[], result='', pause_duration=1, max_retries=3):
    config = configparser.ConfigParser()
    config.read('Config.ini')

    if 'BITRIX' not in config:
        raise ValueError("Section 'BITRIX' manquante dans le fichier 'Config.ini'")

    BITRIX_DOMAIN = config.get('BITRIX', 'BITRIX_DOMAIN')
    BITRIX_TOKEN = config.get('BITRIX', 'BITRIX_TOKEN')

    if not BITRIX_DOMAIN or not BITRIX_TOKEN:
        raise ValueError("Domaine et/ou token Bitrix manquants dans le fichier 'Config.ini'")

    params_str = '&'.join(params)
    BitrixUrl = f'https://{BITRIX_DOMAIN}/rest/1/{BITRIX_TOKEN}/{content}?{params_str}&start='

    listIds = []
    listNom = []
    listCOMPANY_ID = []
    listDATE_CREATE = []
    listEmails = []
    listPhones = []  
    listidd = []
    listFonction = []

    url = BitrixUrl + "0"
    while True:
        try:
            response = requests.get(url)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            raise ValueError(f"Erreur lors de la récupération depuis Bitrix : {e}")

        data = response.json()
        if 'result' in data:
            contacts = data['result']
            for contact in contacts:
                if result == 'email':
                    if 'EMAIL' in contact:
                        email = contact['EMAIL'][0]['VALUE']
                        listEmails.append(email)
                elif result == 'title':
                    listNom.append(contact.get('NAME', ''))
                    listIds.append(contact.get('ID', ''))
                    listCOMPANY_ID.append(contact.get('COMPANY_ID', ''))
                    listDATE_CREATE.append(contact.get('DATE_CREATE', ''))
                    listEmails.append(contact['EMAIL'][0]['VALUE'] if 'EMAIL' in contact else '')
                    listPhones.append(contact['PHONE'][0]['VALUE'] if 'PHONE' in contact else '') 
                    listidd.append(contact.get('UF_CRM_1686296776', ''))
                    listFonction.append(contact.get('POST', ''))
                else:
                    return "Erreur avec les données attendues. Merci de vérifier la variable 'result'."

        if 'next' in data:
            url = BitrixUrl + str(data['next'])
        else:
            break

    if result == 'email':
        df_btx = pd.DataFrame({'email': listEmails})
    elif result == 'title':
        df_btx = pd.DataFrame({
            'ID': listIds,
            'Noms': listNom,
            'ID Entreprise': listCOMPANY_ID,
            "Email": listEmails,
            "Mobile bitrix": listPhones,
            "Date de creation": listDATE_CREATE,
            "IDS": listidd,
            "Fonction": listFonction
        })
    else:
        return "Erreur avec les données attendues. Merci de vérifier la variable 'result'."

    return df_btx



df_Bitrix_contact2 = CrmBitrix(
    content='crm.contact.list.json',
    params=[
        'select[]=ID',
        'select[]=NAME',
        'select[]=COMPANY_ID',
        'select[]=EMAIL',
        'select[]=PHONE', 
        'select[]=DATE_CREATE',
        'select[]=UF_CRM_1686296776',
        'select[]=POST',
    ],
    result="title"
)

# Vérification
print(df_Bitrix_contact2)


In [ ]:
df_Bitrix_contact2.to_csv('contacts_yav.csv', index=False, encoding='utf-8')
df_Bitrix_contact2

In [16]:
import pandas as pd
import requests
defaultCrmProspectListUrl = ""

listUrls = []
listeId = []
listID = []
listValuesource = []
listIDd = []

datas = {}

responses = requests.get(defaultCrmProspectListUrl)
        
def testedvs():
    if responses.status_code == 200:
        datas = responses.json()
        companyfield = datas['result'][0]['LIST']

        for source in companyfield:
            listID.append(source['ID'])
            listValuesource.append(source['VALUE'])
           
            

            df_btx = pd.DataFrame(list(zip(listID, listValuesource)),
                              columns = ["IDS",'Statut'])

        return df_btx

In [17]:
dfdds = testedvs()
dfdds

,IDS,Statut
0,46,NRP
1,48,A rappeler
2,78,Demande de plaquette
3,68,Rdv positionné sur acuity
4,74,Rdv Non venu
5,76,Rdv effectué moyen
6,70,Rdv effectué intéressé
7,72,Mail de relance à envoyer
8,86,A envoyé une demande de financement
9,88,Financements fonctionnels


In [ ]:
df_Bitrixyavs = df_Bitrix_contact2.merge(dfdds, how='left')
df_Bitrixyavs.drop_duplicates()
df_Bitrixyavs.to_csv('contactyavlist.csv', index=False, encoding='utf-8')
df_Bitrixyavs

<span style="color: black; font-weight: bold; font-size: 50px;">La liste des contact(YAV) -> Contact (Freshworks)</span>

In [ ]:
import csv
import requests
import json
import time

BATCH_SIZE = 900
SLEEP_TIME = 3600
API_KEY_FRESHWORK = ''
DOMAIN_FRESHWORK = ''
BASE_URL = f'https://{DOMAIN_FRESHWORK}.myfreshworks.com/crm/sales/'


def get_contact_statuses(api_key, domain):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Token token={api_key}'
    }
    url = f"https://{domain}.myfreshworks.com/crm/sales/api/selector/contact_statuses"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        status_dict = {}
        for item in data.get("contact_statuses", []):
            name = item.get("name", "").strip().lower()
            status_id = item.get("id")
            lifecycle_id = item.get("lifecycle_stage_id")
            if name and status_id and lifecycle_id:
                status_dict[name] = {
                    "status_id": status_id,
                    "lifecycle_stage_id": lifecycle_id
                }
        print("Statuts récupérés depuis l’API :")
        for name, val in status_dict.items():
            print(f"- {name} → status_id: {val['status_id']} | lifecycle_stage_id: {val['lifecycle_stage_id']}")
        return status_dict
    else:
        print(f"Erreur récupération statuts : {response.status_code}")
        return {}


def detect_unknown_statuts(contacts, status_info_dict):
    inconnus = set()
    for contact in contacts:
        statut = str(contact.get("Statut", "")).strip().lower()
        if statut and statut not in status_info_dict:
            inconnus.add(statut)
    if inconnus:
        print("\nStatuts inconnus dans le CSV :")
        for s in inconnus:
            print(f"{s}")
    else:
        print("Tous les statuts du CSV sont reconnus.\n")


def extract_contact_info(row, status_info_dict):
    status_name = str(row.get("Statut", "")).strip().lower()
    status_info = status_info_dict.get(status_name)

    if not status_info:
        print(f"Statut non reconnu : '{status_name}', utilisation par défaut.")
        status_info = status_info_dict.get("non contacté", {
            "status_id": None,
            "lifecycle_stage_id": None
        })

    bitrix_id = str(row.get("ID", "")).strip()
    last_name_raw = str(row.get("Noms", "")).strip()
    last_name = "" if last_name_raw == "." else last_name_raw 
    email = str(row.get("Email", "")).strip()
    mobile = str(row.get("Mobile bitrix", "")).strip()
    fonction = str(row.get("Fonction", "")).strip()

    if not bitrix_id:
        print(f"Ignoré : ID Bitrix manquant")
        return None
    if not (last_name or email):
        print(f"Ignoré (aucun nom ni email) → ID: {bitrix_id}")
        return None
    if not mobile:
        print(f"Ignoré (pas de mobile) → ID: {bitrix_id}")
        return None

    return {
        "last_name": last_name,
        "email": email,
        "mobile_number": mobile,
        "custom_field": {
            "cf_id_bitrix": bitrix_id,
            "cf_date_de_creation": str(row.get("Date de creation", "")),
            "cf_fonction": fonction,
            "status_name": status_name
        },
        "contact_status_id": status_info["status_id"],
        "lifecycle_stage_id": status_info["lifecycle_stage_id"]
    }



def create_contact(api_key, domain, contact_data):
    url = f"https://{domain}.myfreshworks.com/crm/sales/api/contacts"
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Token token={api_key}'
    }
    return requests.post(url, headers=headers, json={"contact": contact_data})


def main():
    try:
        with open('contactyavlist.csv', 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            bitrix_contacts = list(reader)
    except Exception as e:
        print("Erreur : fichier CSV non trouvé ou invalide.")
        return

    if not bitrix_contacts:
        print("Aucun contact trouvé dans le fichier CSV.")
        return

    status_info_dict = get_contact_statuses(API_KEY_FRESHWORK, DOMAIN_FRESHWORK)
    if not status_info_dict:
        print("Aucune correspondance de statut trouvée.")
        return

    detect_unknown_statuts(bitrix_contacts, status_info_dict)

    total_contacts = len(bitrix_contacts)
    batches = [bitrix_contacts[i:i + BATCH_SIZE] for i in range(0, total_contacts, BATCH_SIZE)]

    for batch_index, batch in enumerate(batches, start=1):
        print(f"\nTraitement du lot {batch_index}/{len(batches)}")

        for contact in batch:
            contact_info = extract_contact_info(contact, status_info_dict)
            if not contact_info:
                continue

            bitrix_id = contact_info['custom_field']['cf_id_bitrix']
            try:
                response = create_contact(API_KEY_FRESHWORK, DOMAIN_FRESHWORK, contact_info)
                if response.status_code == 201:
                    print(f"Contact créé → ID: {bitrix_id}")
                else:
                    print(f"Échec création → ID: {bitrix_id} | Status: {response.status_code}")
                    try:
                        print("Détail de l'erreur :", json.dumps(response.json(), indent=2, ensure_ascii=False))
                    except:
                        print("Impossible de lire le détail JSON")
            except Exception as e:
                print(f"Exception sur contact {bitrix_id} : {e}")

        if batch_index < len(batches):
            print(f"Pause de {SLEEP_TIME} secondes avant le prochain lot...")
            time.sleep(SLEEP_TIME)

    print("\nTraitement terminé.")


if __name__ == "__main__":
    main()
